In [1]:
from astropy.table import Table, MaskedColumn, Column
import numpy as np
import astropy.units as u
w51e_dendro_matched_catalog = '/home/t.yoo/w51/w51_frag/dendrogram/dendro_w51e_matched.fits'
w51n_dendro_matched_catalog = '/home/t.yoo/w51/w51_frag/dendrogram/dendro_w51n_matched.fits'

def exponential_to_scientific(num, format='%.2e'):
    """Converts a number in Python's exponential notation to scientific notation."""
    num_str =str(format%num)
    # Check if the number is already in scientific notation
    if 'e' not in num_str.lower():
        return np.nan

    mantissa, exponent = num_str.lower().split('e')
    mantissa = float(mantissa)
    exponent = int(exponent)
    return r"$%s\times10^{%s}$"%(mantissa, exponent)

def exponential_to_scientific2(num, num2, format1='%.2e', format2='%.2e'):
    """Converts a number in Python's exponential notation to scientific notation."""
    num_str1 =str(format1%num)
    num_str2 =str(format2%num2)

    # Check if the number is already in scientific notation
    if 'e' not in num_str1.lower():
        return np.nan

    mantissa, exponent = num_str1.lower().split('e')
    mantissa = float(mantissa)
    exponent = int(exponent)


    if 'e' not in num_str2.lower():

        return np.nan

    mantissa2, exponent2 = num_str2.lower().split('e')
    mantissa2 = float(mantissa2)
    exponent2 = int(exponent2)
    return r"$%s\times10^{%s}\pm %s\times10^{%s}$"%(mantissa, exponent, mantissa2, exponent2)


def make_latex_table1(input, output1, output2, num=30, dist_pc=5400, format='ascii.latex'):
    table = Table.read(input)
    totalnum = len(table['b6_xsky'])
    ra_tab_arr = []
    dec_tab_arr = []
    id_arr = []
    group_new = np.zeros(totalnum, dtype='U30')
    peak_flux_b6 = np.empty(totalnum, dtype='U30')
    peak_flux_b3 = np.empty(totalnum, dtype='U30')
    flux_b6 = np.empty(totalnum, dtype='U50')
    flux_b3 = np.empty(totalnum, dtype='U50')
    constT_mass = np.empty(totalnum, dtype='U50')
    MBBT_mass = np.empty(totalnum, dtype='U50')
    deconvolved_b6 = np.empty(totalnum, dtype='U50')
    deconvolved_b3 = np.empty(totalnum, dtype='U50')
    alpha = np.empty(totalnum, dtype='U30')



    table['peak_flux_b6'].format='%.2e'
    table['peak_flux_b3'].format='%.2e'
    table['alpha'].format='%.1f'
    table['alphaerr'].format='%.1f'
    table['flux_b6'].format='%.2e'
    table['flux_b3'].format='%.2e'
    table['flux_err_b6'].format='%.2e'
    table['flux_err_b3'].format='%.2e'
    table['temp'].format='%d'
    table['constT_mass'].format='%.2e'
    table['constT_mass_err'].format='%.2e'
    table['MBBT_mass'].format='%.2e'
    table['MBBT_mass_err'].format='%.2e'
    table['deconvolved_major_b6'].format='%.2e'
    table['deconvolved_minor_b6'].format='%.2e'
    table['deconvolved_major_b3'].format='%.2e'
    table['deconvolved_minor_b3'].format='%.2e'
    table['b6_pa'].format='%.1f'
    table['b3_pa'].format='%.1f'

    deconvolved_major_b6_mask = table['deconvolved_major_b6'].mask
    deconvolved_major_b3_mask = table['deconvolved_major_b3'].mask
    peak_flux_b6_mask = table['peak_flux_b6'].mask
    peak_flux_b3_mask = table['peak_flux_b3'].mask
    flux_b6_mask = table['flux_b6'].mask
    flux_b3_mask = table['flux_b3'].mask
    constT_mass_mask = table['constT_mass'].mask
    MBBT_mass_mask = table['MBBT_mass'].mask
    alpha_mask = np.zeros(totalnum, dtype=bool)

    for i in range(totalnum):
        if table['b6_xsky'][i] > 0:
            ra_tab = table['b6_xsky'][i]
            dec_tab = table['b6_ysky'][i]
        else:
            ra_tab = table['b3_xsky'][i]
            dec_tab = table['b3_ysky'][i]
        ra_tab_arr.append(ra_tab)
        dec_tab_arr.append(dec_tab)
        id_arr.append(i)

        if table['group'][i]==0:
            group_new[i] = 'dust-dominated'
        elif table['group'][i]==1:
            group_new[i] = 'possibly ff'
        elif table['group'][i]==2:
            group_new[i] = 'ff'
  
        peak_flux_b6[i] = exponential_to_scientific(table['peak_flux_b6'][i])
        peak_flux_b3[i] = exponential_to_scientific(table['peak_flux_b3'][i])
        flux_b6[i] = exponential_to_scientific2(table['flux_b6'][i], table['flux_err_b6'][i])
        flux_b3[i] = exponential_to_scientific2(table['flux_b3'][i], table['flux_err_b3'][i])
        constT_mass[i] = exponential_to_scientific2(table['constT_mass'][i], table['constT_mass_err'][i])
        MBBT_mass[i] = exponential_to_scientific2(table['MBBT_mass'][i], table['MBBT_mass_err'][i])
        major_b6_au = table['deconvolved_major_b6'][i]*3600*dist_pc
        minor_b6_au = table['deconvolved_minor_b6'][i]*3600*dist_pc
        major_b3_au = table['deconvolved_major_b3'][i]*3600*dist_pc
        minor_b3_au = table['deconvolved_minor_b3'][i]*3600*dist_pc

        if np.isfinite(major_b6_au):
            deconvolved_b6[i] = r'$%d\times %d$'%(major_b6_au, minor_b6_au)
        else:
            deconvolved_b6[i] = np.nan
        if np.isfinite(major_b3_au):
            deconvolved_b3[i] = r'$%d\times %d$'%(major_b3_au, minor_b3_au)
        else:
            deconvolved_b3[i] = np.nan
      
        alpha[i] = r'$%.1f\pm%.1f$'%(table['alpha'][i], np.abs(table['alphaerr'][i]))
        if np.isfinite(table['alpha'][i]) or table['alpha'][i]>-99:
            alpha_mask[i] = False
        else:
            alpha_mask[i] = True

        if deconvolved_b6[i]==r'$0\times 0$':
            deconvolved_major_b6_mask[i] = True
            print('mask')
        if deconvolved_b3[i]==r'$0\times 0$':
            deconvolved_major_b3_mask[i] = True

    
    ra_tab_arr = np.round(ra_tab_arr, 5)
    dec_tab_arr = np.round(dec_tab_arr, 5)

    deconvolved_b3_col = MaskedColumn(deconvolved_b3, name=r'$a_{\rm 3mm}\times b_{\rm 3mm}$', mask= deconvolved_major_b3_mask, fill_value='-')
    deconvolved_b6_col = MaskedColumn(deconvolved_b6, name=r'$a_{\rm 1.3mm}\times b_{\rm 1.3mm}$', mask= deconvolved_major_b6_mask)
    
    peak_flux_b6_col = MaskedColumn(peak_flux_b6, name=r'$F_{\rm peak, 1.3mm}$', mask=peak_flux_b6_mask)
    peak_flux_b3_col = MaskedColumn(peak_flux_b3, name=r'$F_{\rm peak, 3mm}$', mask=peak_flux_b3_mask)
    flux_b6_col = MaskedColumn(flux_b6, name=r'$F_{\rm int, 1.3mm}$', mask=flux_b6_mask)
    flux_b3_col = MaskedColumn(flux_b3, name=r'$F_{\rm int, 3mm}$', mask=flux_b3_mask)
    constT_mass_col = MaskedColumn(constT_mass, name=r'$M_{\rm lowlim, constT}$', mask=constT_mass_mask)
    MBBT_mass_col = MaskedColumn(MBBT_mass, name=r'$M_{\rm lowlim, MBBT}$', mask=MBBT_mass_mask)
    alpha_col = MaskedColumn(alpha, name=r'$\alpha$',mask=alpha_mask)
    group_new_col = MaskedColumn(group_new, name='Spectral Index Group', mask=alpha_mask)

    table_new1 = Table([id_arr, np.array(ra_tab_arr)*u.deg, np.array(dec_tab_arr)*u.deg, peak_flux_b6_col, peak_flux_b3_col, alpha_col, group_new_col,],
                       # flux_b6_col, flux_b3_col,table['temp'],constT_mass_col, MBBT_mass_col,deconvolved_b6_col, deconvolved_b3_col, table['b6_pa'], table['b3_pa']], 
                     
                        #deconvolved_b6, deconvolved_b3, table['b6_pa'], table['b3_pa']],
                      names=('ID', 'RA', 'Dec',  r'$F_{\rm peak, 1.3mm}$', r'$F_{\rm peak, 3mm}$', r'$\alpha$', 'Spectral Index Group'), 
                           #  r'$F_{\rm int, 1.3mm}$', r'$F_{\rm int, 3mm}$',r'$T_{\rm dust, MBB}$',r'$M_{\rm lowlim, constT}$',r'$M_{\rm lowlim, MBBT}$',r'$a_{\rm 1.3mm}\times b_{\rm 1.3mm}$', r'$a_{\rm 3mm}\times b_{\rm 3mm}$', r'$\theta_{\rm 1.3mm}$', r'$\theta_{\rm 3mm}$' )
                             #, ', r'$M_{\rm lowlim, MBBT}$', 
                        #       deconvolved_b6, deconvolved_b3, table['b6_pa'], table['b3_pa'])
                      )
    
    table_new2 = Table([id_arr, flux_b6_col, flux_b3_col,table['temp'].filled(-99999999),constT_mass_col, MBBT_mass_col.filled(-99999999),deconvolved_b6_col.filled('-'), deconvolved_b3_col.filled('-'), table['b6_pa'], table['b3_pa']], 
                     
                        #deconvolved_b6, deconvolved_b3, table['b6_pa'], table['b3_pa']],
                      names=('ID', r'$F_{\rm int, 1.3mm}$', r'$F_{\rm int, 3mm}$',r'$T_{\rm dust, MBB}$',r'$M_{\rm lowlim, constT}$',r'$M_{\rm lowlim, MBBT}$',r'$a_{\rm 1.3mm}\times b_{\rm 1.3mm}$', r'$a_{\rm 3mm}\times b_{\rm 3mm}$', r'$\theta_{\rm 1.3mm}$', r'$\theta_{\rm 3mm}$' )
                             #, ', r'$M_{\rm lowlim, MBBT}$', 
                        #       deconvolved_b6, deconvolved_b3, table['b6_pa'], table['b3_pa'])
                      )
    
    print(table['temp'])
    table_truncated1 = table_new1[:num]
    


    table_truncated1.write(output1, format=format, overwrite=True)

    table_truncated2 = table_new2[:num]
    


    table_truncated2.write(output2, format=format, overwrite=True)




In [2]:
make_latex_table1(w51e_dendro_matched_catalog, '/home/t.yoo/w51/w51_frag/dendro_w51e_matched1.tex','/home/t.yoo/w51/w51_frag/dendro_w51e_matched2.tex',num=30)
make_latex_table1(w51n_dendro_matched_catalog, '/home/t.yoo/w51/w51_frag/dendro_w51n_matched1.tex', '/home/t.yoo/w51/w51_frag/dendro_w51n_matched2.tex', dist_pc=5400,num=30)


mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
temp
----
  19
  --
  --
  28
  64
  14
   7
  --
  61
  --
 ...
  --
  --
  --
  --
  --
  --
  --
  --
  --
  --
  --
Length = 137 rows
mask
mask
mask
mask
mask
mask
mask
temp
----
  25
  --
  --
  --
  51
  --
  --
  32
  --
  --
 ...
  --
  --
  --
  --
  --
  --
  --
  --
  --
  --
  --
Length = 74 rows


/scratch/local/59672102/ipykernel_1806348/1985915112.py:21: UserWarning: Warning: converting a masked element to nan.
  num_str1 =str(format1%num)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:22: UserWarning: Warning: converting a masked element to nan.
  num_str2 =str(format2%num2)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:9: UserWarning: Warning: converting a masked element to nan.
  num_str =str(format%num)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:21: UserWarning: Warning: converting a masked element to nan.
  num_str1 =str(format1%num)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:22: UserWarning: Warning: converting a masked element to nan.
  num_str2 =str(format2%num2)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:9: UserWarning: Warning: converting a masked element to nan.
  num_str =str(format%num)


In [3]:
make_latex_table1(w51e_dendro_matched_catalog, '/home/t.yoo/w51/w51_frag/dendrogram/dendro_w51e_matched1.fits','/home/t.yoo/w51/w51_frag/dendro_w51e_matched2.fits',num=128, format='fits')
make_latex_table1(w51n_dendro_matched_catalog, '/home/t.yoo/w51/w51_frag/dendrogram/dendro_w51n_matched1.fits', '/home/t.yoo/w51/w51_frag/dendro_w51n_matched2.fits', dist_pc=5400,num=72, format='fits')

/scratch/local/59672102/ipykernel_1806348/1985915112.py:21: UserWarning: Warning: converting a masked element to nan.
  num_str1 =str(format1%num)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:22: UserWarning: Warning: converting a masked element to nan.
  num_str2 =str(format2%num2)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:9: UserWarning: Warning: converting a masked element to nan.
  num_str =str(format%num)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:21: UserWarning: Warning: converting a masked element to nan.
  num_str1 =str(format1%num)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:22: UserWarning: Warning: converting a masked element to nan.
  num_str2 =str(format2%num2)
/scratch/local/59672102/ipykernel_1806348/1985915112.py:9: UserWarning: Warning: converting a masked element to nan.
  num_str =str(format%num)


mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
mask
temp
----
  19
  --
  --
  28
  64
  14
   7
  --
  61
  --
 ...
  --
  --
  --
  --
  --
  --
  --
  --
  --
  --
  --
Length = 137 rows
mask
mask
mask
mask
mask
mask
mask
temp
----
  25
  --
  --
  --
  51
  --
  --
  32
  --
  --
 ...
  --
  --
  --
  --
  --
  --
  --
  --
  --
  --
  --
Length = 74 rows
